# Multi Layer Perceptrons
- CIFAR-10 image classification with MLP


In [0]:
!pip3 install torch torchvision

In [1]:
import numpy as np
import pandas as pd
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
torch.__version__

'1.0.0'

## 1. Import & process dataset
- CIFAR10 dataset can be downloaded by ```torchvision```
  - [torchvision.datasets](https://pytorch.org/docs/stable/torchvision/datasets.html)

In [25]:
from torchvision import datasets
import torchvision.transforms as transforms

train_dataset = datasets.CIFAR10(root = "/", train = True, download = True, transform = transforms.ToTensor())
test_dataset = datasets.CIFAR10(root = "/", train = False, download = True, transform = transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


## 2. Creating MLP model and training

- Create and train MLP with 1 hidden layer

![](http://cs231n.github.io/assets/nn1/neural_net.jpeg)

In [0]:
# create data loaders 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 128, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 128, shuffle = False)

In [0]:
# create MLP with one hidden layer
class net(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(net, self).__init__()
    self.input_dim = input_dim
    self.dense1 = nn.Linear(input_dim, hidden_dim)
    self.relu = nn.ReLU()
    self.dense2 = nn.Linear(hidden_dim, output_dim)
  
  def forward(self, x):
    x = x.view(x.size(0), self.input_dim)
    x = self.dense1(x)
    x = self.relu(x)
    x = self.dense2(x)
    return x

In [0]:
# hyperparameters
DEVICE = torch.device('cuda')
INPUT_DIM = 32 * 32 * 3
HIDDEN_DIM = 50
OUTPUT_DIM = 10
LEARNING_RATE = 1e-3
NUM_EPOCHS = 10

In [0]:
model = net(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM).to(DEVICE)
criterion = nn.CrossEntropyLoss()   # do not need softmax layer when using CEloss criterion
optimizer = torch.optim.RMSprop(model.parameters(), lr = LEARNING_RATE)

In [34]:
# training for NUM_EPOCHS
for i in range(NUM_EPOCHS):
  temp_loss = []
  for (x, y) in train_loader:
    x, y = x.float().to(DEVICE), y.to(DEVICE)
    outputs = model(x)
    loss = criterion(outputs, y)
    temp_loss.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  print("Loss at {}th epoch: {}".format(i, np.mean(temp_loss)))

Loss at 0th epoch: 2.0718233820117646
Loss at 1th epoch: 1.8863357893951105
Loss at 2th epoch: 1.8327037266758093
Loss at 3th epoch: 1.7953610965967788
Loss at 4th epoch: 1.764086505336225
Loss at 5th epoch: 1.7412422986896448
Loss at 6th epoch: 1.7202257174055289
Loss at 7th epoch: 1.6997354250124959
Loss at 8th epoch: 1.6853232405069845
Loss at 9th epoch: 1.6730405244680926


## 3. Evaluation
- Evaluate the trained MLP model with ROC AUC score 
  - Store probability of each instance to a list and compare it with true y label

In [39]:
y_pred, y_true = [], []
with torch.no_grad():
  for x, y in test_loader:
    x, y = x.float().to(DEVICE), y.to(DEVICE)
    outputs = F.softmax(model(x)).max(1)[-1]       # predicted label
    y_true += list(y.cpu().numpy())                # true label
    y_pred += list(outputs.cpu().numpy())   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


In [40]:
# evaluation result
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.369